In [ ]:
import requests
import urllib3
import tqdm.notebook as tqdm
import os
import json 

# Disable InsecureRequestWarning for self-signed certificates
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

LIVE_URL = 'https://127.0.0.1:2999/liveclientdata/allgamedata'
DDRAGON_URL = 'https://ddragon.leagueoflegends.com/cdn/15.4.1/data/en_US/champion/'
MERAKI_URL = 'https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/champions/'

def get_live_data():
    """Fetch live game data with SSL verification disabled."""
    try:
        response = requests.get(LIVE_URL, verify=False)
        return response.json()
    except Exception as e:
        print("Error fetching live data:", e)
        return None

def get_champion_data(live_data):
    """Extract champion names for Blue and Red teams from live data."""
    out = {'Blue': [], 'Red': []}
    for player in live_data.get('allPlayers', []):
        player_champ = player.get('championName')
        team = player.get('team')
        player_team = 'Blue' if team == 'ORDER' else 'Red'
        out[player_team].append(player_champ)
    return out

def get_champion_cooldown(champion_name, recursive=False):
    """
    Fetch champion cooldown data from Meraki.
    Adjusts the champion name for 'Wukong' and skips passive ability.
    """
    try:
        if champion_name == 'Wukong':
            champion_name = 'monkeyKing'
        url = f"{MERAKI_URL}{champion_name}.json"
        response = requests.get(url)
        data = response.json()
        abilities = data.get('abilities', {})
        champ_icon = data.get('icon')  # Champion icon URL

        cooldowns = {'champIcon': champ_icon}
        for spell_key, spell_list in abilities.items():
            if spell_key == 'P':  # Skip passive ability
                continue
            spell = spell_list[0]  # Assuming the first element is needed

            # Process cooldown values
            if 'cooldown' in spell and spell['cooldown']:
                spell_cds = spell['cooldown']['modifiers'][0]['values']
                if all(cd == spell_cds[0] for cd in spell_cds):
                    cooldowns[spell_key] = f"{spell_cds[0]} all ranks"
                else:
                    cooldowns[spell_key] = " / ".join(str(cd) for cd in spell_cds)
            else:
                cooldowns[spell_key] = 'No CD'

            # Process ammo recharge ability if it exists
            if 'rechargeRate' in spell and spell['rechargeRate']:
                recharge_rate = spell['rechargeRate']
                if all(rate == recharge_rate[0] for rate in recharge_rate):
                    recharge_string = f"{recharge_rate[0]} all ranks"
                else:
                    recharge_string = " / ".join(str(rate) for rate in recharge_rate)
                cooldowns[spell_key] += f" <br> recharge: <br> {recharge_string}"
        return cooldowns
    except Exception as e:
        # On error return empty dictionary
        return {}

def get_ddragon_latest_version():
    """Fetch the latest Data Dragon version."""
    try:
        response = requests.get('https://ddragon.leagueoflegends.com/api/versions.json')
        versions = response.json()
        return versions[0] if versions else None
    except Exception as e:
        print("Error fetching Data Dragon version:", e)
        return None

def get_champion_cooldown_ddragon(champion_name):
    """
    Fetch champion cooldown data from Data Dragon.
    Adjusts the champion name for 'Wukong', retrieves the latest version, and then processes the champion spells.
    """
    try:
        if champion_name == 'Wukong':
            champion_name = 'monkeyKing'
        version = get_ddragon_latest_version()
        print(f"Fetching cooldowns for {champion_name} from Data Dragon")
        url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion/{champion_name}.json"
        response = requests.get(url)
        champ_data = response.json().get('data', {}).get(champion_name, {})
        image_full = champ_data.get('image', {}).get('full')
        champ_icon = f"https://ddragon.leagueoflegends.com/cdn/{version}/img/champion/{image_full}"
        print(f"Champion icon: {champ_icon}")

        champ_spells = champ_data.get('spells', [])
        cooldowns = {'champIcon': champ_icon}
        spell_keys = ['Q', 'W', 'E', 'R']
        for index, spell_key in enumerate(spell_keys):
            if index < len(champ_spells):
                spell = champ_spells[index]
                spell_cds = spell.get('cooldown', [])
                if spell_cds:
                    if all(cd == spell_cds[0] for cd in spell_cds):
                        spell_cd_description = f"{spell_cds[0]} all ranks"
                    else:
                        spell_cd_description = " / ".join(str(cd) for cd in spell_cds)
                    cooldowns[spell_key] = spell_cd_description
        return cooldowns
    except Exception as e:
        print(f"Error fetching cooldowns for {champion_name} from Data Dragon:", e)
        return {}

def get_champion_cooldowns(champions):
    """
    For each champion in the provided teams dictionary, fetch their cooldown data.
    If Meraki's data is empty, fallback to Data Dragon, and if needed try a closest match.
    Returns a tuple: (champion_cooldowns, team_champ_dict)
    """
    champion_cooldowns = {}
    team_champ_dict = {}
    if type(champions) is not dict:
        champions = {'Blue': champions, 'Red': []}
    total_champions = len(champions['Blue']) + len(champions['Red'])
    bar = tqdm.tqdm(total=total_champions, desc="Fetching cooldowns", unit="champion")
    for team, champs in champions.items():
        for champ in champs:
            bar.set_description(f"Fetching cooldowns for {champ}")
            team_champ_dict[champ] = team
            meraki_data = get_champion_cooldown(champ)
            if meraki_data:
                champion_cooldowns[champ] = meraki_data
            else:
                champion_cooldowns[champ] = get_champion_cooldown_ddragon(champ)
            # If both data sources return empty data, try finding the closest match
            if not champion_cooldowns[champ]:
                closest_match = find_closest_match(champ, meraki=True)
                closest_match_ddragon = find_closest_match(champ, meraki=False)
                print(f"Error fetching cooldowns for {champ}, trying closest match: {closest_match} and {closest_match_ddragon}")
                if closest_match != closest_match_ddragon:
                    print("Difference in closest match, going with Data Dragon data")
                    cdragon_data = get_champion_cooldown_ddragon(closest_match_ddragon)
                    champion_cooldowns[champ] = cdragon_data
                    print("Data Dragon data:", cdragon_data)
                else:
                    print("No difference in closest match, going with Meraki data")
                    meraki_data = get_champion_cooldown(closest_match)
                    champion_cooldowns[champ] = meraki_data
                    print("Meraki data:", meraki_data)
            bar.update(1)
    bar.close()
    return champion_cooldowns, team_champ_dict

def champion_cooldowns_renamed(champion_cooldowns):
    """
    Rename the keys to match what the player sees in-game.
    """
    out = {}
    for champ, data in champion_cooldowns.items():
        if champ == 'monkeyKing':
            new_name = 'Wukong'
        # Separate name by capital letters
        new_name = ''.join([f" {c}" if c.isupper() else c for c in champ]).strip()
        if champ == 'Aphelios':
            new_Q = "I just can't man, this dude has like 5 different Q abilities"+\
            "\n and it's formatted in such a fucked up way that I just give up"
            data['Q'] = new_Q
        out[new_name] = data
    return out

def get_all_champion_names(meraki=True):
    """
    Fetch a list of champion names.
    If meraki is True, use the Meraki endpoint; otherwise, use Data Dragon.
    """
    if meraki:
        try:
            all_champs_url = "https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/champions.json"
            response = requests.get(all_champs_url)
            data = response.json()
            return list(data.keys())
        except Exception as e:
            print("Error fetching champion list from Meraki:", e)
            return []
    else:
        try:
            version_response = requests.get('https://ddragon.leagueoflegends.com/api/versions.json')
            versions = version_response.json()
            latest_version = versions[0] if versions else None
            all_champs_url = f"https://ddragon.leagueoflegends.com/cdn/{latest_version}/data/en_US/champion.json"
            response = requests.get(all_champs_url)
            data = response.json().get('data', {})
            return list(data.keys())
        except Exception as e:
            print("Error fetching champion list from Data Dragon:", e)
            return []

def find_closest_match(champ_name, meraki=True):
    """
    Find the champion name closest to the provided string using the Levenshtein distance.
    """
    all_champs_list = get_all_champion_names(meraki=meraki)
    closest_match = None
    closest_distance = float('inf')
    for champ in all_champs_list:
        distance = levenshtein_distance(champ_name.lower(), champ.lower())
        if distance < closest_distance:
            closest_distance = distance
            closest_match = champ
    return closest_match

def levenshtein_distance(a, b):
    """Compute the Levenshtein distance between strings a and b."""
    dp = [[0] * (len(b) + 1) for _ in range(len(a) + 1)]
    for i in range(len(a) + 1):
        dp[i][0] = i
    for j in range(len(b) + 1):
        dp[0][j] = j

    for i in range(1, len(a) + 1):
        for j in range(1, len(b) + 1):
            if a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i][j - 1], dp[i - 1][j]) + 1
    return dp[len(a)][len(b)]

def generate_manifest(version):
    manifest = {
        "currentVersion": version,
        "currentFile": f"champion_cooldowns_{version}.json",
    }
    manifest_fp = f"data/manifest.json"
    with open(manifest_fp, 'w') as f:
        json.dump(manifest, f, indent=4)
    print(f"Manifest saved to {manifest_fp}.")


In [2]:
import pandas as pd

In [3]:
if __name__ == "__main__":
    current_version = get_ddragon_latest_version()
    fp = f"data/champion_cooldowns_{current_version}.json"
    
    if not os.path.exists(fp):
        print(f"File {fp} does not exist, creating it now.")
        all_champs_list = get_all_champion_names(meraki=True)
        if all_champs_list:
            champion_cooldowns, team_champ_dict = get_champion_cooldowns(all_champs_list)
            champion_cooldowns = champion_cooldowns_renamed(champion_cooldowns)
            # Dump to JSON file
            with open(fp, 'w') as f:
                json.dump(champion_cooldowns, f, indent=4)
            print(f"Cooldown data saved to {fp}.")
        else:
            print("No live data available.")
        
    else:
        print(f"File {fp} already exists.")

    # Generate manifest for the current version
    generate_manifest(current_version)


File data/champion_cooldowns_15.22.1.json does not exist, creating it now.


Fetching cooldowns:   0%|          | 0/171 [00:00<?, ?champion/s]

Fetching cooldowns for Yunara from Data Dragon
Champion icon: https://ddragon.leagueoflegends.com/cdn/15.22.1/img/champion/Yunara.png
Cooldown data saved to data/champion_cooldowns_15.22.1.json.
Manifest saved to manifest.json.
